## Load Mojo Kernels

In [1]:
import os, numpy, torch
from pathlib import Path
from max.torch import CustomOpLibrary

assert torch.cuda.is_available()
op_dir = os.path.abspath('operations')

## Simple `add_one` Operation

In [32]:
op_lib = CustomOpLibrary(Path(op_dir))
add_one = op_lib.my_add_constant[{"value": 1}]

In [12]:
import time

def torch_add_one(inputs):
    return inputs + 1

def mojo_add_one(inputs):
    outputs = torch.zeros_like(inputs)
    add_one(outputs, inputs)
    return outputs

for device in ["cpu", "cuda"]:
    for op in [torch_add_one, mojo_add_one]:
        x = torch.zeros(1024, device=device)
        x = op(x) # warm-up
        start = time.perf_counter()
        for _ in range(1000):
            x = op(x)
        end = time.perf_counter()
        print(op.__name__, device, x, end - start)

torch_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.0034156449837610126
mojo_add_one cpu tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.]) 0.13251558190677315
torch_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.0074024859350174665
mojo_add_one cuda tensor([1001., 1001., 1001.,  ..., 1001., 1001., 1001.], device='cuda:0') 0.11501298600342125


## Different MatMul Algorithms

In [2]:
from max.driver import Accelerator, accelerator_count, Tensor
import torch
M, K, N = 8, 8, 16 # debug
M, K, N = 4096, 6144, 2048 # real-world scale
device = Accelerator()
torch_A = torch.randn(M, K)
torch_B = torch.randn(K, N)
torch_result = (torch_A @ torch_B).numpy()
A = Tensor.from_numpy(torch_A.numpy()).to(device)
B = Tensor.from_numpy(torch_B.numpy()).to(device)

Build and test executing the CUDA graph for our MatMul kernel:

In [12]:
from max.graph import Graph, TensorType, DeviceRef, ops
def build_graph(session, algorithm):
    print('building cuda graph for', algorithm)
    with Graph("matmul_graph",
               input_types=[
                   TensorType(dtype=A.dtype, shape=A.shape, device=DeviceRef.from_device(device)),
                   TensorType(dtype=B.dtype, shape=B.shape, device=DeviceRef.from_device(device))
               ],
               custom_extensions=[Path(op_dir)]) as graph:
        A_value, B_value = graph.inputs
        output = ops.custom(
            name="my_matmul",
            device=DeviceRef.from_device(device),
            values=[A_value, B_value],
            out_types=[
                TensorType(dtype=A.dtype, shape=[
                        A_value.tensor.shape[0], B_value.tensor.shape[1]
                    ], device=DeviceRef.from_device(device))
            ],
            parameters={"algorithm": algorithm},
        )
        graph.output(output[0].tensor)
    print('loading cuda graph...')
    return session.load(graph) # compile the graph

from max.engine import InferenceSession
session = InferenceSession(devices=[device])
graph =  build_graph(session, "block_tiling_vectorized")
mojo_result = graph.execute(A, B)[0].to_numpy()
print("test run:\n", mojo_result, end="\n\n")
print("reference:\n", torch_result)

building cuda graph for block_tiling_vectorized
loading cuda graph...
test run:
 [[-28.625193 -46.688335 -88.989105 ...  36.06502  -35.287117  54.835403]
 [ 56.312782 104.198204  38.341404 ... 208.98502  -19.512184 -20.594614]
 [ 65.77984  -69.62101  -40.321484 ... -61.907494  41.775375 -57.322456]
 ...
 [-32.4395    37.203293 -27.763094 ... -41.12329  -29.607489 -56.234642]
 [153.34306   50.26897  -18.373732 ...  24.011692  48.713627 -70.761635]
 [ 10.674924  18.719488 -86.469864 ... -56.90059   89.86108  113.085945]]

reference:
 [[-28.62547   -46.688408  -88.98922   ...  36.065216  -35.28707
   54.835396 ]
 [ 56.312798  104.198135   38.341343  ... 208.9851    -19.512243
  -20.594603 ]
 [ 65.77982   -69.620766  -40.32139   ... -61.907475   41.775356
  -57.3225   ]
 ...
 [-32.43953    37.203293  -27.762918  ... -41.123325  -29.60744
  -56.23472  ]
 [153.3428     50.269035  -18.373772  ...  24.0117     48.7136
  -70.76151  ]
 [ 10.6749935  18.719475  -86.46975   ... -56.900684   89.861

Verify kernel results:

In [13]:
assert numpy.allclose(mojo_result, torch_result, rtol=0, atol=0.005)
for row in range(torch_result.shape[0]):
    if numpy.allclose(torch_result[row], mojo_result[row], rtol=0, atol=0.005): continue
    print('mismatch row:', row)

The tiled kernels can be visualized using the [matmul_visualization.mojo](./matmul_visualization.mojo) and `matmul_visualization_gui/*` scripts.

Here is an example screenshot:
![](./block_tiled_matrix_multiplication.png)

Run a complete benchmark for different algorithms:

In [14]:
import time
for algo in ["naive", "coalescing", "tiled", "tiled_register", "block_tiling", "block_tiling_vectorized"]:
    graph =  build_graph(session, algo)
    record = dict(torch=0, mojo=0)
    sampels = 5
    for _ in range(sampels):
        torch_A = torch.randn(M, K).to('cuda:0')
        torch_B = torch.randn(K, N).to('cuda:0')
        A = Tensor.from_numpy(torch_A.cpu().numpy()).to(device)
        B = Tensor.from_numpy(torch_B.cpu().numpy()).to(device)
        # torch
        torch.cuda.synchronize()
        begin = time.perf_counter()
        torch_result = torch_A @ torch_B
        torch.cuda.synchronize()
        record['torch'] += (time.perf_counter() - begin) / sampels
        # mojo
        torch.cuda.synchronize()
        begin = time.perf_counter()
        mojo_result = graph.execute(A, B)
        torch.cuda.synchronize()
        record['mojo'] += (time.perf_counter() - begin) / sampels
        assert numpy.allclose(mojo_result[0].to_numpy(), torch_result.cpu().numpy(), rtol=0, atol=0.005)
    print(algo, record)

building cuda graph for naive
loading cuda graph...
naive {'torch': 0.02188459157478064, 'mojo': 0.5217830974142998}
building cuda graph for coalescing
loading cuda graph...
coalescing {'torch': 0.013295324030332269, 'mojo': 0.1321177643723786}
building cuda graph for tiled
loading cuda graph...
tiled {'torch': 0.013247651234269141, 'mojo': 0.11083593759685754}
building cuda graph for tiled_register
loading cuda graph...
tiled_register {'torch': 0.01325707440264523, 'mojo': 0.40571665316820144}
building cuda graph for block_tiling
loading cuda graph...
block_tiling {'torch': 0.013388729374855756, 'mojo': 1.172405601781793}
building cuda graph for block_tiling_vectorized
loading cuda graph...
block_tiling_vectorized {'torch': 0.013264452223666014, 'mojo': 0.22294112339150163}


## Reference
[1] https://github.com/modular/modular/blob/main/examples/custom_ops/kernels/matrix_multiplication.mojo

[2] https://docs.modular.com/max/tutorials/custom-ops-matmul